# To-Do List
Database

Image Quality - How to check

GT quality - Mean pixels strength, pixel mean value

Images w/ metadata
Metadata - Key Frames - Categories

Import batch of frames - Curate Frames - Check Ground Truth - Generate Metadata - Output
WandB
Flask Front End
Image Output

scalability

wandb - monitor quality

# Import FastAPI, FFmpeg, uvicorn, and JAAD

In [1]:
%%capture
!pip install fastapi ffmpeg uvicorn JAAD python-multipart tensorflow-gpu scikit-image imutils wandb

from fastapi import FastAPI, File, UploadFile
from skimage.metrics import structural_similarity as compare_ssim


from google.cloud import storage
import nest_asyncio, uvicorn, os

import cv2, wandb

# Log In to WandB

In [36]:
wandb.login(relogin=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Set up ResNet50 Model

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]


2021-09-14 05:15:05.409202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-14 05:15:05.415900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-09-14 05:15:05.416800: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-09-14 05:15:05.417581: I tensorflow/core/platform/cpu_f

# Set Up Google Cloud Parameters


## Set Up Google Cloud Project and Model Location

In [4]:
project = 'mlops-content1' # Cloud Project Name
location = 'james-mlops-capstone' # Model Storage Bucket
model_dir = 'model'

## Create Storage Bucket

In [5]:
storage_client = storage.Client.from_service_account_json('gcp-key.json')

bucket = storage_client.bucket(location)

## Double Check Cloud Bucket (Development Code Only)

In [6]:
%%capture
blobs = storage_client.list_blobs(location)
for blob in blobs:
    print(blob.name)

# WandB Functions

In [7]:
def init_wandb(project_name):
   global wandb_project
   wandb_project = str(project_name)
   wandb.init(project=wandb_project, sync_tensorboard=True)
   return True

# Set File Location

In [8]:
def set_folder_location(in_location):
    global location 
    location = str(in_location)
    global bucket
    bucket = storage_client.bucket(location)
    return True

# Split Video to Frames and Upload

## Download Video to Local Instance

In [9]:
def download_video(video_name):
    print("Downloading: " + str(video_name))
    blob = bucket.blob(video_name)
    blob.download_to_filename(video_name)

## Break down video to frames

In [10]:
import ffmpeg, cv2, shutil
def split_video_frames(video_name):
    print("Splitting: " + str(video_name))
    folder = video_name[:-4]
    try:
        shutil.rmtree(str(folder))
    except:
        pass
    os.mkdir(str(folder))
    
    video_capture = cv2.VideoCapture(str(video_name))
    saved_frame_name = 1

    while True:
        print("Frame: " + format(saved_frame_name, '05d'), end="\r")
        success, frame = video_capture.read()

        if success:
            cv2.imwrite(f"{str(folder)}/frame{format(saved_frame_name, '05d')}.png", frame)
            saved_frame_name += 1
        else:
            break
    print("Done                       ")

## Upload Video Frames

In [11]:
def upload_frames_from_folder(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    #files=files[1:]
    
    print("Uploading Frames")
    for i in range(len(files)):
        print(files[i] + "             ", end="\r")
        blob = bucket.blob(folder_name + "/" + files[i])
        blob.upload_from_filename(folder_name + "/" + files[i])
        
    print("Done Uploading               ", end="\r")

## SSIM Compare Video Frames for Novel Frames

## Remove Blurry Images from Set

### Calculate Blurriness using Laplacian

In [12]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

### Remove Blurry Images

In [32]:
import numpy as np

def remove_blurry_images(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    
    blurriness = np.zeros(len(files))
    
    print("Calculating Average Blurriness")
    for i in range(len(files)):
        print(files[i] + "             ", end="\r")
        img=cv2.imread(folder_name+'/'+files[i])
        img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blurriness[i]=variance_of_laplacian(img_gray)
        wandb.log({'Individual Laplacian': blurriness[i]})
    
    median_blur = np.median(blurriness)
    wandb.log({'Batch Median Laplacian': median_blur})
    print("Median Blur (Laplacian Variance): " + str(median_blur))
    blur_cutoff = median_blur*1.05 #+ ((1-average_blur)*0.1)
    print("Blur Cutoff (Laplacian Variance): " + str(blur_cutoff))
    
    print("Removing Noisy Images")
    
    count = 0
    for i in range(len(files)):
        if blurriness[i] > blur_cutoff:
            print("Deleting " + files[i] + " - Laplacian Noisiness: " + str(blurriness[i]))
            os.remove(folder_name+'/'+files[i])
            count += 1
    blur_ratio = count/len(files)
    wandb.log({'Noisy Frame Ratio': blur_ratio})
    print("Done Checking Frames                  ")


init_wandb(location)
download_video("video_0001.mp4")
split_video_frames("video_0001.mp4")
remove_blurry_images("video_0001")

Batch Median Laplacian,nan
_runtime,6
_timestamp,1631597665
_step,0


Batch Median Laplacian,
_runtime,▁
_timestamp,▁
_step,▁


Downloading: video_0001.mp4
Splitting: video_0001.mp4
Done                       
Calculating Average Blurriness
Median Blur (Laplacian Variance): 77.81829009660333
Blur Cutoff (Laplacian Variance): 81.7092046014335
Removing Noisy Images
Deleting frame00251.png - Laplacian Noisiness: 85.68482653416737
Deleting frame00252.png - Laplacian Noisiness: 85.41787768928081
Deleting frame00253.png - Laplacian Noisiness: 83.07925679900359
Deleting frame00254.png - Laplacian Noisiness: 83.13897686625147
Deleting frame00255.png - Laplacian Noisiness: 81.84813386730562
Deleting frame00256.png - Laplacian Noisiness: 82.11850187274577
Deleting frame00257.png - Laplacian Noisiness: 82.2835965696587
Deleting frame00258.png - Laplacian Noisiness: 83.364253039411
Deleting frame00259.png - Laplacian Noisiness: 82.04787127097715
Deleting frame00260.png - Laplacian Noisiness: 81.92665541227478
Deleting frame00261.png - Laplacian Noisiness: 81.89983780544829
Deleting frame00262.png - Laplacian Noisiness: 82.

# Deduplicate Similar Frames

## Calculate Similarity Between Images

In [22]:
def compare_images(image1, image2):
    image_gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image_gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    diff, _ = compare_ssim(image_gray1, image_gray2, full=True)
    return diff

## Remove Duplicates

In [ ]:
def remove_duplicates(folder_name):
    files=sorted(os.listdir(str(folder_name)))
    files=files[1:]
    print("Removing Duplicate and Highly Similar Frames\nCalculating Frame Similarities")
    
    diff = np.zeros(len(files)-1)    
    
    for i in range(len(files)-1):
        image1 = cv2.imread(folder_name+'/'+files[i])
        image2 = cv2.imread(folder_name+'/'+files[i+1])
        diff[i] = compare_images(image1, image2)
        wandb.log({'Individual Frame Similarities': diff[i]})
        print(str(diff[i]), end="\r")
    
    median_diff = np.median(diff)
    wandb.log({'Batch Median Frame Similarity': median_diff})
    
    diff_cutoff = median_diff*1.05
    
    if diff_cutoff < 0.95:
        diff_cutoff = 0.95
        
    print("Similarity Cutoff (OpenCV Compare Images): " + str(diff_cutoff))
    print("Removing Duplicate Images")
    
    count = 0
    for i in range(len(diff)):
        if diff[i] > 0.99:
            print("Deleting " + files[i] + " - Similarity: " + str(diff[i]), end="\r")
            os.remove(folder_name+'/'+files[i])
            wandb.log({'Duplicates Similarity': diff})
            count += 1
        
    duplicate_ratio = count/len(files)
    wandb.log({'Batch Duplicate Remove Ratio': duplicate_ratio})
    print("Done Checking Frames, " + str(count) + " frames removed.")

    
init_wandb(location)
remove_duplicates("video_0001")

Individual Frame Similarities,0.89337
_runtime,184
_timestamp,1631598904
_step,576
Batch Median Frame Similarity,0.99256


Individual Frame Similarities,▇▇▆▆▆▆▅▄▃▄▇██████████████████████▇█▇▆▃▂▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Batch Median Frame Similarity,▁


Removing Duplicate and Highly Similar Frames
Calculating Frame Similarities


# Full Video Analysis and Upload

In [ ]:
def clean_video(video_name):
    video_name = str(video_name)
    folder_name = str(video_name)[:-4]
    init_wandb(location)
    download_video(video_name)
    split_video_frames(video_name)
    remove_blurry_images(folder_name)
    remove_duplicates(folder_name)
    upload_frames_from_folder(folder_name)

# Analyze Entire Bucket

In [20]:
def clean_entire_bucket():
    blobs = storage_client.list_blobs(location)
    for blob in blobs:
        print(blob.name)

    
    #for file in files:
    #    name = file.strip("gs://"+location)
    #    print("Cleaning Video: " + str(name))
    #    clean_video(name)
clean_entire_bucket()

# Test Script

In [21]:
#clean_video("video_0002.mp4")

# Import Labels

In [22]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd

with open('jaad_database.pkl', 'rb') as pickle_file:
    pickle_data = pickle.load(pickle_file)

labels = pd.DataFrame(data=pickle_data)
labels['video_0001']

num_frames                                                           600
width                                                               1920
height                                                              1080
ped_annotations        {'0_1_3b': {'old_id': 'pedestrian1', 'frames':...
vehicle_annotations    {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: ...
traffic_annotations    {'road_type': 1, 0: {'ped_crossing': 0, 'ped_s...
Name: video_0001, dtype: object

# FastAPI Deployment

In [23]:
app = FastAPI()

@app.on_event("startup")
def start_wandb():
   init_wandb(location)
   return {'message': ('Weights and Balances Started as project: ' + wandb_project)}

@app.get('/')
def index():
    return {'message': 'This is the homepage of the model, add \'/docs\' to the end of the URL to access FastAPI to make predictions with the model'}

@app.get('/set_gcp_location')
def set_gcp_location(string_input):
    set_folder_location(str(string_input))
    return {'message': ('GCP Location Set to: ' + location)}

@app.get('/clean_single_video')
async def single_clean(string_input):
    clean_video(str(string_input))
    return {'message': ('Video: ' + str(string_input) + ' cleaned and uploaded to gs://' + location)}

@app.get('/clean_bucket')
async def full_clean():
    clean_entire_bucket()
    return {'message': ('Bucket: ' + location + ' cleaned and uploaded to gs://' + location)}

                
#@app.post('/predict_single')
#async def predict_api(file: UploadFile = File(...)):
#    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
#    if not extension:
#        return "Image must be jpg or png format!"
#    image = read_imagefile(await file.read())
#    prediction = run_predict_single(image)
#    prediction = str(prediction)
#    print(prediction)
#    return prediction

# Run Deployment

In [24]:
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

INFO:     Started server process [12189]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/uvicorn/protocols/http/h11_impl.py", line 136, in handle_events
    event = self.conn.next_event()
  File "/opt/conda/lib/python3.7/site-packages/h11/_connection.py", line 443, in next_event
    exc._reraise_as_remote_protocol_error()
  File "/opt/conda/lib/python3.7/site-packages/h11/_util.py", line 76, in _reraise_as_remote_protocol_error
    raise self
  File "/opt/conda/lib/python3.7/site-packages/h11/_connection.py", line 425, in next_event
    event = self._extract_next_receive_event()
  File "/opt/conda/lib/python3.7/site-packages/h11/_connection.py", line 367, in _extract_next_receive_event
    event = self._reader(self._receive_buffer)
  File "/opt/conda/lib/python3.7/site-packages/h11/_readers.py", line 68, in maybe_read_from_IDLE_client
    raise LocalProtocolError("illegal request li

INFO:     99.147.232.13:59508 - "GET / HTTP/1.1" 200 OK
INFO:     99.147.232.13:59508 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     99.147.232.13:59508 - "GET /docs HTTP/1.1" 200 OK
INFO:     99.147.232.13:59508 - "GET /openapi.json HTTP/1.1" 200 OK
Downloading: video0002.mp4
INFO:     99.147.232.13:64116 - "GET /clean_single_video?string_input=video0002.mp4 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/uvicorn/protocols/http/h11_impl.py", line 373, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/opt/conda/lib/python3.7/site-packages/uvicorn/middleware/proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "/opt/conda/lib/python3.7/site-packages/fastapi/applications.py", line 208, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/conda/lib/python3.7/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/conda/lib/python3.7/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/opt/conda/lib/python3.7/site-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/opt/conda/lib/python3.7/site-pa

Downloading: video_0002.mp4
Splitting: video_0002.mp4
mkdir: cannot create directory ‘video_0002’: File exists
Calculating Average Blurriness
Average Blur (Laplacian Variance): 118.65879611907178
Blur Cutoff (Laplacian Variance): 124.59173592502538
Removing Blurry Images
Deleting frame0001.png - Blurriness: 160.5361510474856
Deleting frame0002.png - Blurriness: 160.55138396183614
Deleting frame0003.png - Blurriness: 155.81093768205062
Deleting frame0004.png - Blurriness: 154.56900771992852
Deleting frame0005.png - Blurriness: 153.0689992985889
Deleting frame0006.png - Blurriness: 154.5938834731643
Deleting frame0007.png - Blurriness: 154.38932801427694
Deleting frame0008.png - Blurriness: 153.55330497709718
Deleting frame0009.png - Blurriness: 152.0936585101511
Deleting frame0010.png - Blurriness: 152.25109189121366
Deleting frame0011.png - Blurriness: 145.9799455856427
Deleting frame0012.png - Blurriness: 153.18484643373256
Deleting frame0013.png - Blurriness: 137.8901494502126
Deleti

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [12189]
